In [1]:
from src.code_gen.code_generator import CCodeGenerator

generator = CCodeGenerator()

In [2]:
code = generator('''  
    function fib(n) => if (n == 0 | n == 1) 1 else fib(n-1) + fib(n-2);

    print(fib(5));
    '''
, False)

print(code)

Object* function_fib (Object* p0);

Object* ifElseBlock0(Object* p0);

Object* ifElseBlock0(Object* p0) {
   if(*((bool*)getAttributeValue(boolOr(((Object* (*)(Object*, Object*))getMethodForCurrentType(p0, "equals", 0))(p0, createNumber(0)), ((Object* (*)(Object*, Object*))getMethodForCurrentType(p0, "equals", 0))(p0, createNumber(1))), "value"))) {
      return createNumber(1);
   }
   else {
      return numberSum(function_fib(copyObject(numberMinus(p0, createNumber(1)))), function_fib(copyObject(numberMinus(p0, createNumber(2)))));
   }
}

Object* function_fib (Object* p0) {
   return ifElseBlock0(p0);
}


int main() {
   function_print(copyObject(function_fib(copyObject(createNumber(5)))));
   return 0; 
}
